# Python RAG Pattern with Semantic Kernel and PgVector

## Azure PostgreSQL Flexible Server - PGVector Setup in Azure

### Running a local database with a container:

- docker pull pgvector/pgvector:pg16
- Then execute:

```bash
docker run --name pgvector16 \
  --restart unless-stopped \
  -p 5432:5432 \
  -v pgdata:/var/lib/postgresql/data \
  -d pgvector/pgvector:pg16
```
- After deployment, connect using psql and type: `CREATE EXTENSION vector;`

### Running a Flexible Server in Azure - Manual Instructions:

- Create a Flexible server instance in the Azure Portal
- After creation, navigate to the Server Parameters pane:
  - Search for azure.extensions
  - Check the `Vector` value
  - Save the changes and wait for the server to deploy
- After deployment, open the instance and navigate to the `Database` panel:
  - Click `Connect` link on the Postgres database
    - Using the Cloud Shell psql, active the vector extension by typing: `CREATE EXTENSION vector;`

Connection string:
- `PG_CONN_STR_PY="postgresql://<user>:<password>@<server>:5432/<database>"`

Useful commands:

- `truncate table public."PYCollection";`

## Setup

### Load required packages

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)
from semantic_kernel.connectors.memory.postgres.postgres_memory_store import (
    PostgresMemoryStore,
)
from dotenv import load_dotenv
import os

COLLECTION_NAME = "PYCollection"
ADA_EMBEDDINGS_SIZE = 1536

### Load the environment variables

In [ ]:
load_dotenv()
endpoint = os.getenv("GPT_OPENAI_ENDPOINT")
api_key = os.getenv("GPT_OPENAI_KEY")
gpt_deployment_name = os.getenv("GPT_OPENAI_DEPLOYMENT_NAME")
conn_str = os.getenv("PG_CONN_STR_PY")
ada_deployment_name = "ada"

### Get a kernel instance configured for text completions and embeddings

In [ ]:
kernel = sk.Kernel()
azure_chat_service = AzureChatCompletion(deployment_name=gpt_deployment_name, endpoint=endpoint, api_key=api_key)
azure_text_embedding = AzureTextEmbedding(deployment_name=ada_deployment_name, endpoint=endpoint, api_key=api_key)
kernel.add_chat_service("chat_completion", azure_chat_service)
kernel.add_text_embedding_generation_service("ada", azure_text_embedding)

mem_store = PostgresMemoryStore(conn_str,ADA_EMBEDDINGS_SIZE,1,3)
if await mem_store.does_collection_exist(COLLECTION_NAME):
    await mem_store.delete_collection(COLLECTION_NAME)
    
kernel.register_memory_store(memory_store=mem_store)
kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "text_memory")
print("Kernel is ready to use")

## Ingestion

### Read the files and chunk them by paragraph

In [ ]:
def read_file(file: str)->str:
    with open(file, "r") as f:
        return f.read()
    
def ingest_content(path:str):
    import os
    chunks = []
    files = os.listdir(path)
    for f in files:
        if f.endswith("water.txt"):            
            content = read_file("data/"+f)
            paragraphs = content.split("\n\n")
            l = len(paragraphs)
            id = 1
            for p in paragraphs:
                lid = f"{f}-{l}-{id}"
                c = {"id":lid,"chunk":p,"file":f}
                chunks.append(c)
                id += 1
    return chunks

chunks = ingest_content("data")

### Save the chunks and embeddings in the vector database

In [ ]:
async def populate_memory(kernel: sk.Kernel, chunks: list) -> None:
    for chunk in chunks:
        await kernel.memory.save_information(collection=COLLECTION_NAME, id=chunk["id"], text=chunk["chunk"], description=chunk["file"])

await populate_memory(kernel, chunks)

## Grounding

### Find memories based on query, and collect the text in the memories to augment the prompt

In [ ]:
async def search_memory_examples(kernel: sk.Kernel, question: str, limit: int=3, relevance=0.75) -> list:
    results = await kernel.memory.search(COLLECTION_NAME, question,limit,relevance)
    return results


## Build a context from the text chunks in the memories

In [ ]:
question = "What is the chemical composition of water?"
results = await search_memory_examples(kernel, question)
prompt_context = "Context: \"\"\"\n"

for result in results:
    prompt_context += f"Text:\n{result.text}\nSource:\n{result.description}\n"
    
prompt_context += "\"\"\""

## Process Prompt & Completion

### Create a SK function

In [ ]:
promptTemplate = "{{$input}}\n\nContext: ===\n{{$context}}\n===\nAdd a source reference to the end of each sentence. e.g. Apple is a fruit [reference1.pdf][reference2.pdf]. Use only the provided text."
rag_function = kernel.create_semantic_function(prompt_template=promptTemplate, max_tokens=500, temperature=0.3)

skf_context = kernel.create_new_context()
skf_context["input"] = question
skf_context["context"] = prompt_context

### Submit the Prompt and print the results

In [ ]:
result = await rag_function(context=skf_context)
print(f"user:\n{question}\n\nassistant:\n{result}")